In [1]:
import numpy as np
import scipy as sp
import timeit
import matplotlib.pyplot as plt
from hat_support import *
from hat_solution import *

### Consumer Problem 

The Consumer Problem is given by 

\begin{align}
    \max \; &E_{0} \sum_{t=0}^{\infty} \beta^t u(c_{it}, n_{it})\\
    \text{s.t.} \; &c_{it} + a_{i t+1} = z_{it}n_{it}w_{t} + R_{t}a_{it}\\
    &a_{it} \geq - \phi
\end{align}

The first order conditions (assuming an interior solution) are

\begin{align}
    &u_{c}(c_{it}, n_{it}) = \beta E_{t} \left[R_{t+1} u_{c}(c_{it+1}, n_{it+1})\right],\\
    &u_{c}(c_{it}, n_{it}) z_{it} w_{t} = - u_{n}(c_{it}, n_{it}).
\end{align}

### Separable Power (POW) Utility

$$u(c, n) = \frac{c^{1-\sigma}}{1-\sigma} - \psi \frac{n^{1+\gamma}}{1+\gamma}$$

The first order conditions (assuming an interior solution) are

\begin{align}
    &c_{it}^{-\sigma} = \beta E_{t} \left[ R_{t+1} c_{it+1}^{-\sigma}\right] ,\\
    &c_{it}^{-\sigma} z_{it} w_{t} = \psi n_{it}^{\gamma}.
\end{align}

The full Euler equation also takes the constraint into account

\begin{align}
    c_{it}^{-\sigma} = \max \left\{\beta E_{t} \left[ R_{t+1} c_{it+1}^{-\sigma}\right], (z_{it}n_{it}w_{t} + R_{t}a_{it} - \phi)^{-\sigma} \right\}
\end{align}

### Endogenous Gridpoint Method

In the stationary equilibrium we have $R_{t} = R$ and $w_{t} = w$ for all $t$. We can solve for the policy functions using the following algorithm:

1. Set a grid for assets $a$ and labor productivity $z$. 
2. Guess a policy function for consumption, $g_c(z,a)$. In the case without labor supply you can guess the asset policy function directly, but here it is easier to guess the policy function for consumption. 
3. Set up a Coleman operator that maps from consumption policy to consumption policy. Specifically, the Coleman operator does the following:
    1. To find the endogenous gridpoints for assets suppose that tomorrow's assets $\hat{a'}$ lie on the asset grid. Using the policy funtion we find $c' = g_c(z',\hat{a'})$. Then we the find endogenous gridpoints $\hat{c}$, $\hat{n}$, $\hat{a}$ for every gridpoint $(z,\hat{a'})$ from the first order conditions assuming an interior solution: 
    \begin{align*}
        \hat{c} &= \left(\beta E \left[ R g_{c}(z', \hat{a'})^{-\sigma}\right]\right)^{-\frac{1}{\sigma}}\\
        \hat{n} &= \left(\frac{1}{\psi} \hat{c}^{-\sigma} z w\right)^{\frac{1}{\gamma}}\\
        \hat{a} &= \frac{1}{R} \left(\hat{c} + \hat{a'} - w \hat{n} z \right)
    \end{align*}
    2. $\hat{a}$ is today's assets given that tomorrow's assets will be the point $\hat{a'}$ on the asset grid. The endogenous gridpoints $\hat{a}$ lie off the asset grid and we need to find tomorrow's assets given that today's assets lie on the asset grid, so we have to interpolate. For a fixed $z$ we have the endogenous gridpoints for assets on the horizontal axis and the points in the asset grid on the vertical axis, since those are next period's assets. Suppose $\left(\hat{a}_{0}, \hat{a}_{1}\right)$ are the coordinates on the horizontal axis that lie to the left and the right of the asset gridpoint $\hat{a}$ and $\left(\hat{a'}_{0}, \hat{a'}_{1}\right)$ are the associated coordinates on the vertical axis. Using linear interpolation:
    \begin{align*}
        a^{'} = \hat{a}_{0} \left(1- \frac{\hat{a} - \hat{a'}_{0}}{\hat{a'}_{1}-\hat{a'}_{0}}\right) + \hat{a}_{1} \left(\frac{\hat{a} - \hat{a'}_{0}}{\hat{a'}_{1}-\hat{a'}_{0}}\right).
    \end{align*}
    In python I can easily do this with the ``numpy.interp`` function: ``numpy.interp(agrid, endogrid_a[:, z_i], agrid)``.
    Now we have an $a^{'}$ associated with gridpoint $(z,\hat{a})$. If $a^{'}\leq \phi$ the consumer is constrained and the policy functions have to satisfy the following two equations for all states $(z, \hat{a})$:

    \begin{align}
        &c^{-\sigma} = znw + R a - \phi,\\
        &c^{-\sigma} z w = \psi n^{\gamma}.
    \end{align}
    
    These equations can be solved numerically for $c$, which will be the new consumption policy for the constrained consumer. 

    If $a^{'}>\phi$ we have to interpolate again to find the new consumption policy. This is because $a'$ lies off the asset grid. This time, for a fixed $z'$, the asset grid is on the horizontal axis and the endogenous grid for consumption is on the vertical axis.  Suppose $\left(a_{0}, a_{1}\right)$ are the coordinates on the horizontal axis that lie to the left and the right of the point $a^{'}$ and $\left(\hat{c}_{0}, \hat{c}_{1}\right)$ are the associated coordinates on the vertical axis. For each $a^{'}$ we compute 

    \begin{align*}
        c = \hat{c}_{0} \left(1- \frac{a^{'} - a_{0}}{a_{1}-a_{0}}\right) + \hat{c}_{1} \left(\frac{a^{'} - a_{0}}{a_{1}-a_{0}}\right).
    \end{align*}
    
     In python I do this using: ``numpy.interp(ap[:, z_i], agrid, endogrid_c[:, z_i])``. This gives us the new policy function for consumption.

4. Find the fixed point of the Coleman operator. This is the consumption policy function $g_c$. Next, we can find the other policy functions:

    \begin{align*}
        g_n &= \min \left\{\max \left\{ \left(\frac{1}{\psi} g_c^{-\sigma} z w\right)^{\frac{1}{\gamma}}, 0 \right\}, 1 \right\}\\
        g_a &= \max \left\{ w g_n z + R a - g_c, \phi \right\}
    \end{align*}

### Stationary equilibrium

In my code I find the interest rate $R$ that clears the asset market. 

### Value function

We can also find the value function. The optimal value function satisfies

\begin{align}
	V(a,z) = u(c, n) + \beta E_{t}[V(a,z)].
\end{align}

Let $U$ be a matrix whose elements are the period utility values $u(c, n)$ at each gridpoint $(a, z)$. These values can be computed from the policy functions. The conditional expectation on the right-hand side can be computed using the transition matrix $\Gamma$. This yields a linear system that can be solved for the discretized value function $V$:

\begin{align}
    (I - \beta \Gamma)vec(V) = vec(U).	
\end{align}

### Perfect-Foresight Transitions

Suppose that there is a steady state with TFP equal to $TFP_{0}$. In period $t=0$ news arrives that TFP follows the path $\{TFP_{t}\}_{t=0}^{\tau}$. I fix a $\hat{T}>\tau$ and assume that the economy will be back at steady state with TFP equal to $TFP_{0}$ in period $\hat{T}$. I use a root finder to find the path of interest rates $\{R_{t}\}$ that clears the asset market in every period. That is $A_{t} - K_{t} \approx 0$ for all $t$. I follow the following steps to find $\{A_{t}\}_{t=0}^{T}$ and $\{K_{t}\}_{t=0}^{T}$:

* Backward Step
    * Compute the stationary equilibrium in  period $\hat{T}$ assuming that TFP will be equal to $TFP_{0}$. Store policy function $g_{T}(a,z)$.
    * Iterate backwars using the Coleman Operator to find policy functions $\{g^{a}_{t}(a,z)\}_{t=0}^{T-1}$, $\{g^{n}_{t}(a,z)\}_{t=0}^{T-1}$, and $\{g^{c}_{t}(a,z)\}_{t=0}^{T-1}$. 
* Forward Step
    * Before $t=0$, the economy was at it's steady state and distribution of assets was the stationary distribution $\lambda^{*}$. Therefore, $\lambda_{0} = T_{-1, 0} \lambda^{*} = \lambda^{*}$. Given the path of $TFP$, we computed new policy rules which give rise of a new set of transition matrices $\{T_{t, t+1}\}_{t=0}^{t=T-1}$. We can use these transition matrices to find the asset distributions over the path: $\lambda_{t+1} = T_{t, t+1} \lambda_{t}$.  Then I compute aggregate capital $\{K_{t}\}_{t=0}^{T}$ and assets $\{A_{t}\}_{t=0}^{T}$. 
    
  

In [2]:
β = 0.96        # Discount Rate
σ = 2           # EIS
ψ = 100         # Labor Disutility Parameter, calibrated so that SS N is approx 0.3
γ = 2           # Inverse of Frisch Elasticity 
α = 0.36        # Output Elasticity of Capital
δ = 0.08        # Depreciation Rate
φ = 0           # Borrowing Constraint

# Set up grids
# ρ = 0.6 and  σ_z = 0.3
log_zgrid, P, estar = tauchen(2, 0.6, 0.3)
zgrid = np.exp(log_zgrid)
agrid = np.linspace(φ, 14, 50) #expanding_grid(φ, 14, 3, 100)
agrid_dense = np.linspace(φ, 14, 50) #expanding_grid(φ, 14, 3, 1000)

# Create Model Instance
m = AiyagariModel(β, σ, ψ, γ, α, φ, δ, P)


## Initial guesses

This prints the guess for R, the w computed from the firm's FOCs given the guess for R, and initial guess for the policy function for consumption. Rows are grid points on the asset grid and columns are points on the grid for z. 

In [3]:
R0 = 1 + 0.7*((1/β)-1)
w0 = m.find_w(R0, 1)
gc0 = zgrid*w0*0.28 + R0*agrid[:,np.newaxis]


## Final iteration of the Coleman operator

I find the fixed point of the Coleman operator using anderson. Here I print the final consumption policy. 

In [4]:
%%time 

fun = lambda gc: m.coleman(agrid, zgrid, gc, R0, w0) - gc     
gc = sp.optimize.anderson(fun, gc0)
ga = np.maximum(w0*zgrid + R0*agrid[:,np.newaxis] - gc, φ)


CPU times: user 48.9 ms, sys: 2.56 ms, total: 51.5 ms
Wall time: 49.9 ms


In [5]:
%%time 

T = m.find_transition_matrix(agrid, agrid_dense, zgrid, ga)


CPU times: user 9.33 ms, sys: 939 µs, total: 10.3 ms
Wall time: 9.39 ms


In [7]:
%%time 

λ = m.find_stationary_distribution(T)

CPU times: user 2.61 ms, sys: 1.32 ms, total: 3.92 ms
Wall time: 2.65 ms


In [6]:
%%time 

v = m.find_value_function(agrid, agrid_dense, zgrid, gc, T)


CPU times: user 354 µs, sys: 108 µs, total: 462 µs
Wall time: 434 µs


## Original code

In [ ]:
# Stationary equilibrium
eqlbrm = m.find_equilibrium(agrid, agrid_dense, zgrid, TFP = 1)

The ``find_equilibrium`` method returns an instance of the ``AiyagariEquilibrium`` class and contains
* R: Equilibrium Interest Rate
* w: Equilibrium Wages
* gc, gn, ga: Policy Functions
* T: Transition Matrix
* λ: Stationary Distribution
* A, C, N, X, Y: Aggregate Quantities


In [ ]:
fig, ax = plt.subplots()
ax.plot(agrid, eqlbrm.ga[:,0], label = 'Low labor income')
ax.plot(agrid, eqlbrm.ga[:,2], label = 'Medium labor income')
ax.plot(agrid, eqlbrm.ga[:,4], label = 'High labor income')
ax.set_xlabel('$a_t$', fontsize=12)
ax.set_ylabel('$a_{t+1}$', fontsize=12)
ax.set_title('Policy Function for Assets', fontsize=12)
plt.grid(True)
ax.legend(ncol = 1, fontsize=12)

In [ ]:
fig, ax = plt.subplots()
ax.plot(agrid, eqlbrm.gc[:,0], label = 'Low labor income')
ax.plot(agrid, eqlbrm.gc[:,2], label = 'Medium labor income')
ax.plot(agrid, eqlbrm.gc[:,4], label = 'High labor income')
ax.set_xlabel('$a_t$', fontsize=12)
ax.set_ylabel('$c_{t}$', fontsize=12)
ax.set_title('Policy Function for Consumption', fontsize=12)
plt.grid(True)
ax.legend(ncol = 1, fontsize=12)

In [ ]:
fig, ax = plt.subplots()
ax.plot(agrid, eqlbrm.gn[:,0], label = 'Low labor income')
ax.plot(agrid, eqlbrm.gn[:,2], label = 'Medium labor income')
ax.plot(agrid, eqlbrm.gn[:,4], label = 'High labor income')
ax.set_xlabel('$a_t$', fontsize=12)
ax.set_ylabel('$n_{t}$', fontsize=12)
ax.set_title('Policy Function for Labor', fontsize=12)
plt.grid(True)
ax.legend(ncol = 1, fontsize=12)

In [ ]:
# Reshape stationary distribution and sum over z
λ = eqlbrm.λ.reshape([agrid_dense.size, zgrid.size], order='F').sum(1)

fig, ax = plt.subplots()
ax.plot(agrid_dense, λ)
ax.set_xlabel('$a$', fontsize=12)
plt.grid(True)
ax.set_title('Stationary Distribution', fontsize=12);

In [ ]:
# NIPA 
print(eqlbrm.Y) # Production
print(eqlbrm.C + eqlbrm.X) # Expenditure
print(eqlbrm.w * eqlbrm.N + (eqlbrm.R-1+δ) * eqlbrm.K) # Income

### Perfect-Foresight Transitions


In [ ]:
TFP0 = 1.0
TFP_path = [1.0, 1.0, 1.0, 1.0, 1.0, 0.9]

# Find Transitions
transition_path = m.find_transiton_path(agrid, agrid_dense, zgrid, TFP0, TFP_path, T_hat=20)


In [ ]:
fig, ax = plt.subplots()
ax.plot(transition_path.Y-eqlbrm.Y)
ax.set_xlabel('$t$', fontsize=12)
ax.set_ylabel('$Y-Y^{SS}$', fontsize=12)
ax.set_title('Transition Path for Output', fontsize=12)
plt.grid(True)


In [ ]:
fig, ax = plt.subplots()
ax.plot(transition_path.C-eqlbrm.C)
ax.set_xlabel('$t$', fontsize=12)
ax.set_ylabel('$C-C^{SS}$', fontsize=12)
ax.set_title('Transition Path for Consumption', fontsize=12)
plt.grid(True)


In [ ]:
fig, ax = plt.subplots()
ax.plot(transition_path.N - eqlbrm.N)
ax.set_xlabel('$t$', fontsize=12)
ax.set_ylabel('$N-N^{SS}$', fontsize=12)
ax.set_title('Transition Path for Labor', fontsize=12)
plt.grid(True)


In [ ]:
fig, ax = plt.subplots()
ax.plot(transition_path.X - eqlbrm.X)
ax.set_xlabel('$t$', fontsize=12)
ax.set_ylabel('$X-X^{SS}$', fontsize=12)
ax.set_title('Transition Path for Investment', fontsize=12)
plt.grid(True)


In [ ]:
# NIPA 
print(transition_path.Y) # Production
print(transition_path.C + transition_path.X) # Expenditure
print(transition_path.w * transition_path.N + (transition_path.R-1+δ) * transition_path.K) # Income